# Déclaration de profils d'acheteur - DECP

In [1]:
import requests
import agate

from pathlib import Path
import cchardet as chardet

In [28]:
# get the list of datasets tagged DECP on data.gouv.fr

url = 'https://www.data.gouv.fr/api/1/datasets/?tag=decp&page_size=1000'
r = requests.get(url)
datasets = r.json()['data']

len(datasets)

13

In [29]:
# Download all tabular files in data/ directory, as best as we can

downloaded = []
for d in datasets:
    orga = d['organization']['slug'] if d['organization'] else d['owner']['slug']
    slug = d['slug']
    for r in d['resources']:
        rurl = r['url']
        rid = r['id']
        # ODS style NB: won't work more than once for CKAN
        if 'format=csv' in rurl:
            filename = rurl.split('/')[-3] + '.csv'
        else:
            filename = rurl.split('/')[-1]
        if filename in downloaded:
            print('x existing file %s' % rurl)
            continue
        ext = filename.split('.')[-1]
        if ext not in ['csv']:
            print('x ignored file %s' % rurl)
            continue
        r = requests.get(rurl, allow_redirects=True)
        p = Path('data/%s' % slug)
        p.mkdir(exist_ok=True, parents=True)
        written_filename = '%s.%s' % (rid, ext) 
        with open('%s/%s' % (p, written_filename), 'wb') as f:
            f.write(r.content)
            downloaded.append(filename)
            print('- downloaded file [%s] %s' % (filename, rurl))
print('Done')

- downloaded file [listes-des-marches-publics-du-casvp-a-compter-de-2018.csv] https://opendata.paris.fr/explore/dataset/listes-des-marches-publics-du-casvp-a-compter-de-2018/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
x ignored file https://opendata.paris.fr/explore/dataset/listes-des-marches-publics-du-casvp-a-compter-de-2018/download?format=json&timezone=Europe/Berlin&use_labels_for_header=true
x ignored file https://static.data.gouv.fr/resources/annuaire-des-profils-acheteurs-des-adherents-de-territoires-numeriques-bourgogne-franche-comte/20180831-135821/2018-07-11-recensement-des-profils-dacheteur-ternum-bfc.xlsx
- downloaded file [2018_07_11_Recensement_des_profils_dacheteur_TerNum_BFC.csv] https://www.data.gouv.fr/s/resources/annuaire-des-profils-acheteurs-des-adherents-de-territoires-numeriques-bourgogne-franche-comte/20180719-111604/2018_07_11_Recensement_des_profils_dacheteur_TerNum_BFC.csv
- downloaded file [decp.csv] https://static.data.gouv.fr/reso

In [43]:
def parse_csv(file_path):
    # deactivate type testing, this puts too much constraint on parsing
    # especially for lat/lon columns with commas
    tester = agate.TypeTester(types=(agate.Text, ), limit=0)
    with file_path.open('rb') as f:
        encoding = chardet.detect(f.read()).get('encoding')
    try:
        # /!\ force delimiter
        # TODO maybe open with ; and if only one column, try with ',' or delim
        table = agate.Table.from_csv(file_path, encoding=encoding, sniff_limit=None, column_types=tester, delimiter=';')
    except Exception as e:
        print('[x] CSV parse error for %s (%s)' % (file_path, e))
    else:
        return table

In [31]:
columns = ['siretAcheteur', 'urlProfilAcheteur', 'coordonnnées']
columns_low = [x.lower() for x in columns]

columns_mapping = [
    ('siretAcheteur', 'siret'), 
    ('urlProfilAcheteur', 'url'), 
    ('coordonnnées', 'coordonnnees'), 
]

In [44]:
# parse CSV files: analyze difference between standard schema and real data
# infer a mapping of column names variation to match a maximum of data later

# /!\ this is only an analysis step that helped build `columns_mapping` above
# you do not need to run this when doing a consolidation

p = Path('data')
for child in [x for x in p.iterdir() if x.is_dir()]:
    csvs = list(child.glob('*.csv'))
    for csv in csvs:
        table = parse_csv(csv)
        if table:
            cols = [x.lower() for x in table.column_names]
            missing_pivot = []
            # TODO also match `siret`
            for pivot in ['siretacheteur']:
                if pivot not in cols:
                    missing_pivot.append(pivot)
            if missing_pivot:
                print('Skipping %s for missing pivot %s, cols were %s' % (csv, missing_pivot, cols))
                break
            diff = list(set(cols) - set(columns_low))
            if diff:
                print('DIFF for %s: %s' % (csv, diff))
print('Done.')

[x] CSV parse error for data/marches-publics-experimentation-opendatalocale-de-la-region-bourgogne-franche-comte/1adbf48a-d669-4c22-96f6-fea6a0f2226e.csv (Row 37 has 2 values, but Table only has 1 columns.)
DIFF for data/profil-acheteur/8571413a-3eb4-4007-9d85-bd04d2a2e114.csv: ['coordonnnees']
DIFF for data/donnees-essentielles-de-la-commande-publique-de-la-ca-et-du-cias-bar-le-duc-sud-meuse/98488e76-aa43-4c5d-9745-bbda3841e8d5.csv: ['coordonnnees']
DIFF for data/publication-profil-dacheteur/cd6a8112-e556-42e8-b785-efe8b0ab93be.csv: ['coordonnnees']
DIFF for data/profil-acheteur-1/74d7da4e-c35f-4115-a7dc-4c53c56df4ca.csv: ['coordonnnees']
DIFF for data/declaration-du-profil-acheteur/e99030a3-b258-42a3-9b20-13f0632bb473.csv: ['coordonnnees']
DIFF for data/decp-declaration-profil-acheteur/ee1c011f-3361-4281-92b6-16830a256b9b.csv: ['coordonnnees']
DIFF for data/declaration-d-un-acheteur/526e769f-2dda-46a2-a951-193dcb39a32b.csv: ['coordonnnees']
Skipping data/annuaire-des-profils-acheteur

In [47]:
# use columns_mapping to build a database of unique stations based on (id_station, id_pdc, max(date_maj))

profils = []

def find_by_pivot(row, lines):
    pass

def ifind_in_row_by_col(col, row):
    for key in row.keys():
        if col.lower() == key.lower():
            return row[key]

p = Path('data')

for child in [x for x in p.iterdir() if x.is_dir()]:
    csvs = list(child.glob('*.csv'))
    for csv in csvs:
        table = parse_csv(csv)
        if table:
            table_cols = [x.lower() for x in table.column_names]
            missing_pivot = []
            # TODO also match `siret`
            for pivot in ['siretacheteur']:
                if pivot not in table_cols:
                    missing_pivot.append(pivot)
            if missing_pivot:
                break
            for row in table.rows:
                profil = {}
                for col in columns_mapping:
                    for c in col:
                        if c.lower() in table_cols:
                            # use the standard name for column
                            profil[col[0]] = ifind_in_row_by_col(c, row)
                            break
                # remove empty lines
                if not profil.get('siretAcheteur'):
                    continue
                upatt = 'https://www.data.gouv.fr/fr/datasets/%s/#resource-%s'
                profil['source'] = upatt % (csv.parents[0].stem, csv.stem)
                # normalize siretAcheteur
                profil['siretAcheteur'] = profil['siretAcheteur'].strip().replace(' ', '')
                profils.append(profil)
                
len(profils)

[x] CSV parse error for data/marches-publics-experimentation-opendatalocale-de-la-region-bourgogne-franche-comte/1adbf48a-d669-4c22-96f6-fea6a0f2226e.csv (Row 37 has 2 values, but Table only has 1 columns.)
[x] CSV parse error for data/listes-des-marches-publics-du-casvp-a-compter-de-2018/b6fd2a6a-bfd9-4de9-98bc-e9d671ea70d3.csv (Row 4 has 39 values, but Table only has 22 columns.)


19

In [48]:
# enjoy the results!

import csv

with open('decp-dpa.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns + ['source'], delimiter=';')
    writer.writeheader()
    for b in profils:
        writer.writerow(b)

## debug / test

In [42]:
source = 'data/declaration-du-profil-acheteur/e99030a3-b258-42a3-9b20-13f0632bb473.csv'
table = agate.Table.from_csv(source, encoding="latin-1", delimiter=';')
table.column_names

('siretAcheteur', 'urlProfilAcheteur', 'coordonnnees')